In [1]:
import os
from datetime import datetime,timezone,timedelta

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm
from scipy import signal

import data_filter as dafi
import data_process as dapr
import lib_plot

### define the function

In [2]:
def generate_timeseries(main_path, azimut_mask, elevation_mask, min_height, max_height, t_range):
    frequency = dapr.generate_frequency(min_height=min_height,max_height=max_height)
    signal_ts = {}

    for dt in tqdm(range(0,t_range-1)):
        starttime = datetime(year=2022,month=10,day=31,hour=8,minute=0+dt,second=0,tzinfo=timezone.utc)
        endtime = datetime(year=2022,month=10,day=31,hour=14,minute=0+dt,second=0,tzinfo=timezone.utc)
        deltatime = timedelta(minutes=t_range)

        data_dict = dafi.clean_data(main_path,azimut_mask=azimut_mask,elevation_mask=elevation_mask,sn1_trigger=True)

        split_data_dict = dafi.split_data(data_dict,starttime,endtime,deltatime)
        result_dict = dapr.data_prepare(split_data_dict,frequency=frequency)
        result_dict_1031 = dapr.extract_height_as_max_peak(result_dict,frequency=frequency)

        for satellite_code in result_dict_1031:
            for t,p in zip(result_dict_1031[satellite_code]['time'],result_dict_1031[satellite_code]['power']):
                if t in signal_ts:
                    signal_ts[t] = np.multiply(p,signal_ts[t])
                else:
                    signal_ts[t] = p
    height_ts = {}
    for t in signal_ts:
        p = signal_ts[t]
        index = p==max(p)
        h = frequency[index]
        height_ts[t] = h[0]
    return height_ts

In [6]:
main_path_2 = os.path.abspath('../data/20221031/2/')
main_path_3 = os.path.abspath('../data/20221031/3/')
min_height = 2
max_height = 6
azimut_mask = [270,330]

ts_dict = {}
max_ele_settings = [20,25,30,40,50]
t_range_settings = [15,20,25,30,60]
for ele in max_ele_settings:
    ts_dict[ele] = {}
    for t_range in t_range_settings:
        ts_dict[ele][t_range] = {}
        elevation_mask = [3,ele]
        ts_dict[ele][t_range][2] = generate_timeseries(main_path_2, azimut_mask, elevation_mask, min_height, max_height, t_range)
        ts_dict[ele][t_range][3] = generate_timeseries(main_path_3, azimut_mask, elevation_mask, min_height, max_height, t_range)

100%|██████████| 29/29 [06:49<00:00, 14.13s/it]


In [8]:
import pickle 
with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(ts_dict, f)

In [9]:
with open('saved_dictionary.pkl', 'rb') as f:
    ts_dict = pickle.load(f)